In [5]:
import numpy as np
import json
import sys
import pandas as pd
import os
import glob
import re
import matplotlib.pyplot as plt
import seaborn as sns
from os import listdir
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.datasets import load_digits
from sklearn.model_selection import learning_curve
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.metrics import mean_squared_error
from sklearn.feature_selection import SelectKBest


import warnings
warnings.filterwarnings("ignore")

In [6]:
from sklearn.datasets import make_hastie_10_2
from sklearn.ensemble import GradientBoostingRegressor

In [24]:
sys.path.insert(0, 'src')
from helper import *
from eda import *
from train import *                       
from etl import *

%load_ext autoreload

%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### etl (new readfilerun and gen)

In [19]:
df = readfilerun_simple('data/raw/train_r/20220116T055105_20-100-true-20-100-iperf.csv')
df10 = agg10(genfeat(df))


In [ ]:
300//20

15

In [16]:
[x for x in df.columns if 'a' in x]

['packet_times',
 'packet_sizes',
 'packet_dirs',
 'latency',
 'later_latency',
 'later_loss',
 'total_bytes',
 'total_pkts',
 'mean_tdelta',
 'max_tdelta',
 'max_bytes',
 'total_pkt_sizes',
 'pkt_ratio',
 'time_spread',
 'byte_ratio']

In [30]:

df['group'] = df.index // 20
print(df.shape)
# df.groupby('group').agg(pd.Series.mode)
agg20 = better_agg(df, 20)
agg20

(300, 33)


,total_bytes,max_bytes,max_bytes_std,Proto,1->2Bytes,2->1Bytes,1->2Pkts,2->1Pkts,total_pkts,total_pkts_amin,...,mean_tdelta,mean_tdelta,mean_tdelta,max_tdelta,max_tdelta_amin,max_tdelta_amax,latency,loss,later_latency,later_loss
group,,,,,,,,,,,,,,,,,,,,,
0,421877.333333,22606.222222,8825.437483,6,415000.000000,6877.333333,276.666667,127.222222,403.888889,216,...,2.589280,1.008772,4.562791,41.944444,22,90,20,100,20,100
1,387977.263158,22561.473684,3484.557223,6,382815.789474,5161.473684,255.210526,96.631579,351.842105,252,...,2.809929,1.932653,3.717131,43.473684,43,44,20,100,20,100
2,364914.400000,20441.200000,3506.828010,6,360000.000000,4914.400000,240.000000,91.800000,331.800000,230,...,3.015532,1.912955,4.244541,43.250000,43,45,20,100,20,100
3,370471.157895,21681.263158,4476.812964,6,365605.263158,4865.894737,243.736842,91.105263,334.842105,208,...,3.012020,1.968880,4.487923,43.210526,43,44,20,100,20,100
4,376476.400000,22077.200000,5590.497517,6,371475.000000,5001.400000,247.650000,93.500000,341.150000,211,...,3.036155,1.557662,4.628571,43.400000,43,44,20,100,20,100
5,368127.600000,20070.400000,4804.026977,6,363000.000000,5127.600000,242.000000,96.100000,338.100000,239,...,3.002793,1.812977,4.096639,43.100000,40,44,20,100,20,100
6,307165.894737,18173.263158,3893.975073,6,302763.157895,4402.736842,201.842105,81.947368,283.789474,178,...,3.501010,2.415233,5.457627,43.315789,43,44,20,100,20,100
7,371622.600000,17184.000000,3653.569156,6,365700.000000,5922.600000,243.800000,110.900000,354.700000,258,...,2.895292,1.498423,3.793103,43.300000,39,45,20,100,20,100
8,413536.000000,20377.400000,5037.741562,6,407550.000000,5986.000000,271.700000,112.300000,384.000000,223,...,2.673591,1.809886,4.369369,43.250000,40,45,20,100,20,100


In [ ]:
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import make_scorer

df = pd.read_csv('outputs\\combined_transform.csv')
df


,total_bytes,max_bytes,proto,1->2Bytes,2->1Bytes,1->2Pkts,2->1Pkts,total_pkts,number_ms,pkt_ratio,time_spread,pkt sum,longest_seq,total_pkt_sizes,byte_ratio,loss,latency
0,147546.4,3977.820375,6.0,143716.4,3830.0,96.0,70.9,166.9,36.3,928.661778,905.6,147546.4,5.0,147546.4,1066.509474,100.0,100.0
1,82518.8,2954.589921,6.0,81300.0,1218.8,54.2,22.5,76.7,17.6,1073.823761,901.3,82518.8,5.1,82518.8,1066.509474,100.0,100.0
2,72773.6,2132.137165,6.0,71550.0,1223.6,47.7,22.3,70.0,20.0,1037.201125,920.1,72773.6,4.6,72773.6,1066.509474,100.0,100.0
3,77979.2,2056.799196,6.0,76800.0,1179.2,51.2,22.1,73.3,20.4,1063.096983,916.3,77979.2,4.5,77979.2,1066.509474,100.0,100.0
4,148404.0,2450.035501,6.0,146400.0,2004.0,97.6,37.2,134.8,27.5,1099.955806,960.5,148404.0,6.4,148404.0,1066.509474,100.0,100.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7702,1181395.2,4610.190760,6.0,1167000.0,14395.2,778.0,270.9,1048.9,125.5,1128.508362,911.9,1181395.2,7.6,1181395.2,1105.379896,9000.0,500.0
7703,1352562.4,5613.342138,6.0,1335600.0,16962.4,890.4,326.2,1216.6,140.1,1110.808290,953.7,1352562.4,7.5,1352562.4,1105.379896,9000.0,500.0
7704,2152011.2,113913.653088,6.0,2121928.8,30082.4,1414.8,536.3,1951.1,211.4,1098.366122,664.8,2152011.2,220.4,2152011.2,1105.379896,9000.0,500.0
7705,1420124.8,134999.098851,6.0,1401492.0,18632.8,934.6,324.6,1259.2,151.1,1125.585269,668.7,1420124.8,174.8,1420124.8,1105.379896,9000.0,500.0


In [ ]:
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import make_scorer

df = pd.read_csv('outputs\\combined_transform.csv')
df


,total_bytes,max_bytes,proto,1->2Bytes,2->1Bytes,1->2Pkts,2->1Pkts,total_pkts,number_ms,pkt_ratio,time_spread,pkt sum,longest_seq,total_pkt_sizes,byte_ratio,loss,latency
0,147546.4,3977.820375,6.0,143716.4,3830.0,96.0,70.9,166.9,36.3,928.661778,905.6,147546.4,5.0,147546.4,1066.509474,100.0,100.0
1,82518.8,2954.589921,6.0,81300.0,1218.8,54.2,22.5,76.7,17.6,1073.823761,901.3,82518.8,5.1,82518.8,1066.509474,100.0,100.0
2,72773.6,2132.137165,6.0,71550.0,1223.6,47.7,22.3,70.0,20.0,1037.201125,920.1,72773.6,4.6,72773.6,1066.509474,100.0,100.0
3,77979.2,2056.799196,6.0,76800.0,1179.2,51.2,22.1,73.3,20.4,1063.096983,916.3,77979.2,4.5,77979.2,1066.509474,100.0,100.0
4,148404.0,2450.035501,6.0,146400.0,2004.0,97.6,37.2,134.8,27.5,1099.955806,960.5,148404.0,6.4,148404.0,1066.509474,100.0,100.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7702,1181395.2,4610.190760,6.0,1167000.0,14395.2,778.0,270.9,1048.9,125.5,1128.508362,911.9,1181395.2,7.6,1181395.2,1105.379896,9000.0,500.0
7703,1352562.4,5613.342138,6.0,1335600.0,16962.4,890.4,326.2,1216.6,140.1,1110.808290,953.7,1352562.4,7.5,1352562.4,1105.379896,9000.0,500.0
7704,2152011.2,113913.653088,6.0,2121928.8,30082.4,1414.8,536.3,1951.1,211.4,1098.366122,664.8,2152011.2,220.4,2152011.2,1105.379896,9000.0,500.0
7705,1420124.8,134999.098851,6.0,1401492.0,18632.8,934.6,324.6,1259.2,151.1,1125.585269,668.7,1420124.8,174.8,1420124.8,1105.379896,9000.0,500.0


In [ ]:
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import make_scorer

df = pd.read_csv('outputs\\combined_transform.csv')
df


,total_bytes,max_bytes,proto,1->2Bytes,2->1Bytes,1->2Pkts,2->1Pkts,total_pkts,number_ms,pkt_ratio,time_spread,pkt sum,longest_seq,total_pkt_sizes,byte_ratio,loss,latency
0,147546.4,3977.820375,6.0,143716.4,3830.0,96.0,70.9,166.9,36.3,928.661778,905.6,147546.4,5.0,147546.4,1066.509474,100.0,100.0
1,82518.8,2954.589921,6.0,81300.0,1218.8,54.2,22.5,76.7,17.6,1073.823761,901.3,82518.8,5.1,82518.8,1066.509474,100.0,100.0
2,72773.6,2132.137165,6.0,71550.0,1223.6,47.7,22.3,70.0,20.0,1037.201125,920.1,72773.6,4.6,72773.6,1066.509474,100.0,100.0
3,77979.2,2056.799196,6.0,76800.0,1179.2,51.2,22.1,73.3,20.4,1063.096983,916.3,77979.2,4.5,77979.2,1066.509474,100.0,100.0
4,148404.0,2450.035501,6.0,146400.0,2004.0,97.6,37.2,134.8,27.5,1099.955806,960.5,148404.0,6.4,148404.0,1066.509474,100.0,100.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7702,1181395.2,4610.190760,6.0,1167000.0,14395.2,778.0,270.9,1048.9,125.5,1128.508362,911.9,1181395.2,7.6,1181395.2,1105.379896,9000.0,500.0
7703,1352562.4,5613.342138,6.0,1335600.0,16962.4,890.4,326.2,1216.6,140.1,1110.808290,953.7,1352562.4,7.5,1352562.4,1105.379896,9000.0,500.0
7704,2152011.2,113913.653088,6.0,2121928.8,30082.4,1414.8,536.3,1951.1,211.4,1098.366122,664.8,2152011.2,220.4,2152011.2,1105.379896,9000.0,500.0
7705,1420124.8,134999.098851,6.0,1401492.0,18632.8,934.6,324.6,1259.2,151.1,1125.585269,668.7,1420124.8,174.8,1420124.8,1105.379896,9000.0,500.0


In [ ]:
indexcol = ['total_bytes','max_bytes','2->1Bytes','2->1Pkts','total_pkts', 'total_pkts_min', 'total_pkts_max', 'number_ms', 'pkt_ratio','time_spread', 'time_spread_min','time_spread_max','pkt sum','longest_seq', 'longest_seq_min', 'longest_seq_max','total_pkt_sizes','byte_ratio', 'mean_tdelta', 'max_tdelta']
len(indexcol)

20

['total_bytes',
 'max_bytes',
 '2->1Bytes',
 '2->1Pkts',
 'total_pkts',
 'number_ms',
 'pkt_ratio',
 'time_spread',
 'pkt sum',
 'longest_seq',
 'total_pkt_sizes',
 'byte_ratio']

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import make_scorer

label_col = 'latency'

df = pd.read_csv('outputs\\combined_transform.csv')

indexcol = ['total_bytes','max_bytes','2->1Bytes','2->1Pkts','total_pkts', 'total_pkts_min', 'total_pkts_max', 'number_ms', 'pkt_ratio','time_spread', 'time_spread_min','time_spread_max','pkt sum','longest_seq', 'longest_seq_min', 'longest_seq_max','total_pkt_sizes','byte_ratio', 'mean_tdelta', 'max_tdelta']
len(indexcol)

X_train, X_test, y_train, y_test = train_test_split(
    df[[x for x in indexcol if x in df.columns]], df[label_col])
# print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)
etree = ExtraTreesRegressor(n_estimators=400, n_jobs=4)
etreeft = etree.fit(X_train,y_train)

y_pred3 = etree.predict(X_test)
acc3= mean_squared_error(y_test, y_pred3)

print(f'mse: {acc3}, r2: {etree.score(X_test, y_test)}')
feat_imp = pd.Series(index=[x for x in indexcol if x in df.columns], 
          data=etree.feature_importances_).sort_values(ascending=False)
feat_imp

(5780, 12) (1927, 12) (5780,) (1927,)
mse: 1709.884903347172, r2: 0.9318839253945324


byte_ratio         0.553971
time_spread        0.102298
number_ms          0.075867
pkt_ratio          0.072764
max_bytes          0.032258
2->1Pkts           0.025018
total_pkt_sizes    0.024187
2->1Bytes          0.023896
total_bytes        0.023580
pkt sum            0.023410
total_pkts         0.023066
longest_seq        0.019684
dtype: float64

In [ ]:
acc3

1888.4178210949665

In [ ]:
print(f'mse: {acc3}, r2: {etree.score(X_test, y_test)}')
feat_imp = pd.Series(index=[x for x in indexcol if x in df.columns], 
          data=etree.feature_importances_).sort_values(ascending=False)
feat_imp

mse: 1888.4178210949665, r2: 0.9239026238224772


byte_ratio         0.570085
time_spread        0.099886
pkt_ratio          0.069825
number_ms          0.069792
max_bytes          0.028858
pkt sum            0.024635
total_pkts         0.023934
total_pkt_sizes    0.023690
2->1Bytes          0.022993
2->1Pkts           0.022918
total_bytes        0.022051
longest_seq        0.021332
dtype: float64

In [ ]:
pd.Series(index=[x for x in indexcol if x in df.columns], 
          data=etree.feature_importances_).sort_values(ascending=False)


byte_ratio         0.570085
time_spread        0.099886
pkt_ratio          0.069825
number_ms          0.069792
max_bytes          0.028858
pkt sum            0.024635
total_pkts         0.023934
total_pkt_sizes    0.023690
2->1Bytes          0.022993
2->1Pkts           0.022918
total_bytes        0.022051
longest_seq        0.021332
dtype: float64